In [0]:
from IPython.core.display import HTML
css_file = '../../styles/styles.css'
HTML(open(css_file, "r").read())

# Absolute Photometry

<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span>&nbsp;Learning Objectives</h2>
</div>
</section>

> * How to use the APT tool to extract instrumental magnitudes for many stars
> * Use the ```astroquery``` and ```astropy``` librarys to cross match catalogs on sky position
> * How to compute zeropoints and uncertainties

In the [lecture](http://slittlefair.staff.shef.ac.uk/teaching/phy241/lectures/L07/index.html) we learnt the theory behind precise calibration of photometry, enabling us to put photometric observations onto a standard scale with precisions of better than 1%. In this practical we are going to learn how to produce calibrated photometry using a simplified method. This method does not yield the same accuracy as an analysis using observations of primary or secondary standard stars, combined with the use of colour terms. On the other hand, it is much simpler to apply, and even works to produce calibrated photometry through thin cloud. If you are happy to accept accuracies on the order of a few percent, I recommend using the method described below whenever you need to produce calibrated photometry. 

In this practical we will will produce a colour-magnitude diagrams using the stacked images you made in the previous practical. If you have not yet finished that practical, use the stacked images provided in the ```data``` folder, taken from the open cluster NGC 7789. If you have finished, upload your own stacked images to the ```data``` folder and use them instead.

## Method

In the lecture we saw that, for any star, the difference between the calibrated magnitude, \(m\) and the above-atmosphere instrumental magnitude \(m_{0,i}\) is given by:
\[m = m_{0,i} + m_{\rm zp},\]
where \(m_{\rm zp}\) is known as the zero point. The above-atmosphere instrumental magnitude is given by:
\[m_{0,i} = m_i - kX,\]
where \(m_i = -2.5 \log_{10} \left( N_t/t_{\rm exp} \right)\) is the instrumental magnitude, \(k\) is the extinction coefficient, and \(X\) is the airmass. Therefore:
\[m = m_i - kX + m_{\rm zp}.\]
In other words, if we were to plot the calibrated magnitude \(m\) against instrumental magnitude \(m_i\) for all the stars in our image, we would expect a straight line, with a gradient of one, and an intercept equal to \( -kX + m_{\rm zp} \). This intercept could then be added to all of our instrumental magnitudes to produce calibrated magnitudes. 

What makes this technique possible, is the existence of large sky surveys, which have provided calibrated magnitudes for many, relatively bright stars over a very wide areas of the sky. If your data is covered by one of these surveys, and it provides calibrated magnitudes in the same filter as your data, you can apply this technique relatively quickly.

## Accuracy

The accuracy this technique can achieve is limited by two factors. First of all we are not taking into account any secondary effects, such as 2nd-order extinction, or colour terms. Secondly, our photometry cannot be any more accurate than that in the sky survey we use. Typically, large sky surveys achieve accuracies of a few percent. Ignoring the secondary effects will probably introduce errors on a similar level. If you need calibrated photometry to better than a few percent, you will have to observe standard stars, and apply the rigorous method described in the lecture.

## Steps

Applying this method requires the application of four steps. These steps are:

1. finding the stars in our image, and calculating instrumental magnitudes;
1. matching our stars against a sky-survey, so we know instrumental and calibrated magnitude for many stars;
1. calculating the offset between instrumental and calibrated magnitudes, and applying to all stars in our image.


We will carry out these steps using a combination of a piece of software known called APT, for Aperture Photometry, and Python. As usual we will draw on several third-party Python libraries. I will explain their use, and provide links to detailed documentation. 

# Photometry with APT

We have done aperture photometry [before](http://slittlefair.staff.shef.ac.uk/teaching/phy241/practicals/P05/index.html), using AstroImageJ. Whilst this tool is excellent for relative photometry, and producing light curves, each aperture must be placed by hand, which makes it onerous to use when we want to measure every star in an image. For this reason I will introduce a new software tool, the [Aperture Photometry Tool](http://www.aperturephotometry.org/), or APT for short. 

APT can be installed on Windows, Linux or Mac machines. There is excellent documentation [here](http://www.aperturephotometry.org/aptool/documentation/), and a 35-minute long video tutorial [here](https://www.youtube.com/watch?v=5lXAWfBW_NQ). Should you need to perform aperture photometry on a large number of stars in your course, APT is a good place to start. If it doesn’t meet your needs, or you want to practice your Python - there is a Python library called [photutils](http://photutils.readthedocs.org/en/latest/).

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span>Install APT</h2>
</div>
</section>

> **Windows:** Download the ZIP file from this [link](http://www.aperturephotometry.org/aptool/download/). Once downloaded, **right-click and extract the contents from the ZIP file**. In the folder produced is APT.exe. Run this to start APT.

> **Mac:** Download the DMG file from the link above and open it. Copy the "Aperture Photometry Tool" inside to your Applications folder. You may encounter error messages telling you the DMG file is corrupted. In which case follow the instructions [here](http://www.aperturephotometry.org/aptool/faqs/#q13)



With APT installed, we need to open our data, and change the settings that influence how aperture photometry is done.

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span>Install APT</h2>
</div>
</section>

> Start APT and click **Open Image** to load your astrometrically calibrated image. Start with your 300s V-band stack. We'll need to set the aperture sizes, but first we need to know the FWHM of stars in our image.

> Click on an isolated star in the image to place an aperture around it. Then click on **Radial Profile**. You will see a plot of brightness versus distance from the centre of the star (like the figure below). Also shown on the right is the parameters of a fit to the brightness profile. The FWHM in pixels is shown here

<img src="../../images/rad-prof.png" style="margin: 20px" width=750px/>

> Once you know the FWHM, use it to set the values for your photometry apertures. You should have some experience in this from the relative photometry practicals, but decent starting values are an object aperture radius of 1.5x FWHM, an inner sky radius of 2x FWHM and an outer sky radius of 3x FWHM.

> To set the aperture sizes, click on the big button in the GUI (labelled something like **5/5/0 -> Alter**) to set the aperture parameters. Two windows should appear. The **Major radius** in the larger window is where you enter the object aperture. Click **Apply**. The sky annulus radii are entered in the smaller window. Set these values and click **Apply**. **Close** both windows.


> Lastly, we need to set what model APT will use to measure our photometry Click on **More Settings** and make sure the source algorithm is set to **model 0** (this is straightforward aperture photometry). Your image is likely to be very crowded with stars, and so we need to be careful how we estimate the sky. Many of the stars will have nearby objects contaminating their sky-background annulus. We can minimise the impact of this by setting the sky algorithm is set to **model E** (this will use the *mode* of the values in the sky annulus to measure and subtract the sky background). Click **Apply Settings** and then **Close Window.** 


> **Since the aperture parameters and photometry model you use are fundamental settings needed to replicate your results; make a note of the values you use - they will be important numbers to have for your write-up**.


With the aperture settings finalised, you can perform aperture photometry on any single star by clicking on it. Have a go at this, and look at the values reported in the **Aperture Photometry Results** area of the APT window. However, we want to detect all the stars in the image, and perform aperture photometry on all of them at once.

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span>Creating a Source List</h2>
</div>
</section>

> Click on the **Source List** button at the bottom of the APT screen. The source list tool will appear. Check the button marked **Check box to overlay sources on image** and click **Create Source List** to get the source list creation tool. The important settings for source detection are the **detection threshold**, which sets how bright an object must be to be detected, and the **Radial-profile sigma** which gives APT an idea how large the stars in the image will be. This last value wants to be at least twice the FWHM. 

> Choose settings for these values and click **Create Source List**. The detected sources will be displayed in the window. You want to make sure you are finding even the faint stars, but not detecting things that are not stars! Play with the settings until you are happy and **Close** the source list creation tool.

# Performing Photometry

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span>Measure the stars</h2>
</div>
</section>

> In the source list tool, click **Automatically Process Source List**, to perform aperture photometry with the current aperture settings on all stars in the source list. Close the source list tool window when complete. 

> You can view your results by clicking **List Results** at the bottom of the APT window. Click on **Export Contents** to save your results to a comma-seperated-values (CSV) file that can be used by other programs. In the window that appears click Reset CSV Filename to set the filename to something useful (for example, *V_300s.csv*) and click **Create CSV File**. You will want to upload this CSV file to CoCalc, for use later.

> *It is important to note that the results file is not cleared each time you load a new image, or restart APT. Unless you clear it manually, the old results will still be present.* Before you close the results window, click **Clear Contents** to empty the current results.

# Photometric Calibration

Now we have a CSV table of instrumental magnitudes (and much besides) in it. The next step is photometric calibration. As a reminder, this involves fitting a straight line to a graph of *instrumental magnitude* against *calibrated magnitude*, to find the offset between the two. Equivalently, we can find the average value of the *difference* between the calibrated and instrumental magnitudes for all our stars.

Since we have instrumental magnitudes and sky positions (RA, Dec) for a number of stars, we must find the matching stars in an online catalog of calibrated magnitudes. We will use the [APASS](https://www.aavso.org/apass) catalog; a catalog which combines several other sky surveys to provide data in many filters across much of the sky. Crucially, in this case it includes B and V magnitudes, the two filters used for our photometry.

To perform the cross-matching we will use the [astroquery](https://astroquery.readthedocs.io/en/latest/) Python library, and we will use the [Table](http://docs.astropy.org/en/stable/table/) object from [astropy](http://www.astropy.org) to handle our CSV files. 

In [0]:
from astropy import units as u
from astropy.table import Table
from astroquery.xmatch import XMatch

First, we read in the CSV files we created in APT. To read them in, and make an astropy ```Table``` object is pretty simple. Just replace ```data/V300.csv``` with the location of the CSV file you uploaded to CoCalc:

In [0]:
# REPLACE the filename below with the name of your CSV file!
V300 = Table.read('data/V300.csv')

These astropy tables play nicely with Jupyter notebooks, so you can simply type the name of the table in a code cell to see the table displayed in the browser:

In [0]:
V300

The important columns for our uses are the measured centres of our stars in RA and Dec (**CentroidRA** and **CentroidDec**) and the instrumental magnitude and uncertainty (**Magnitude** and **MagUncertainty**). Note that the magnitude uncertainty is calculated using the CCD signal-to-noise equation we saw in the [lectures](http://slittlefair.staff.shef.ac.uk/teaching/phy241/lectures/L09/index.html).

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span>X-match with APASS</h2>
</div>
</section>

> We need to calibrate our photometry, which will involve comparing our instrumental magnitudes to calibrated magnitudes measured for the same stars. We need to match our detected stars with those catalogued in the sky survey [APASS](https://www.aavso.org/apass). We are looking for stars who's RA and Dec matches to within some radius. A service called **Vizier** hosts online versions of astronomical catalogs, and we can use the ```Xmatch.query``` function to match an astropy table with a table hosted by Vizier using the code below.

> Run the code cell below, and note carefully how we specify the columns that contain RA and Dec in our *local* table, and how we set the maximum distance for a valid match. **II/336/apass9** is the name of the APASS catalog on Vizier. If you need to find the names of other catalogs (perhaps APASS doesn't cover the patch of sky containing your open cluster), you can enter the catalog name in the search box [here](http://vizier.u-strasbg.fr)

> **The code cell below may take a while to run. Be patient...**

In [0]:
xmatch = XMatch.query(cat1=V300, cat2='vizier:II/336/apass9', max_distance=2*u.arcsec, colRA1='CentroidRA', colDec1='CentroidDec')

The result of this query (```xmatch```) is also an astropy ```Table```. It has all the columns from both tables for each valid match. If you want to extract a column and save it into a variable, you can access the ```Table``` like a dictionary. So the code below extracts the **Magnitude** column (which is instrumental magnitude from our APT photometry) and **Vmag** column from UCAC4 and computes the difference.

In [0]:
delta_mag = xmatch['Vmag'] - xmatch['Magnitude']

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span>Plot the difference and find the zero-point</h2>
</div>
</section>

> Plot the difference between instrumental and calibrated magnitude found above, against the calibrated V-band magnitude on the X-axis. You should see something like the figure below:

<img src="../../images/V_zeropoint.png" style="margin: 0px" width=750px/>

In [0]:
# YOUR CODE HERE - MAKE A PLOT LIKE THE ONE ABOVE. DON'T WORRY ABOUT ADDING THE HORIZONTAL LINE

The magnitude difference between instrumental and calibrated magnitude *should* be a constant, which is the value of $kX + m_{\rm zp}$. In the figure above, you can see there are plenty of outlying points, and the bright stars deviate from the constant. The outliers are either stars whose photometry is bad (poor sky estimation, or contaminated by very close stars), or spurious detections (i.e not stars). The deviation of the bright stars is caused because they are [saturated](http://slittlefair.staff.shef.ac.uk/teaching/phy241/lectures/L06/index.html#readout), and so we cannot accurately measure their flux.

Since $m = m_i - kX + m_{\rm zp}$, we can find the value of $-kX + m_{\rm zp}$. - which I'll call the *zeropoint* from now on - by calculating the **median** difference between the instrumental and calibrated magnitude. The median will be robust against the outliers - but we only want to do it for the non-saturated stars!

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span>Calculate the zero-point</h2>
</div>
</section>

> Calculate the median value of ```delta_mag```. **If you have evidence for saturated stars**, use a NumPy *mask* to only calculate the median of stars that are not saturated (see notes on Fancy slicing in the NumPy notebook).

> The median value of ```delta_mag``` is our estimate of $-kX + m_{\rm zp}$ - i.e the value we want to add to our instrumental magnitudes to get a calibrated magnitude. You can add this value to the **Magnitude** column of the V300 table easily using ```V300['Magnitude'] = V300['Magnitude'] + zp```, where ```zp``` is the median value you found.

In [0]:
# YOUR CODE HERE. CALCULATE THE ZEROPOINT AND ADD IT TO THE MAGNITUDE COLUMN OF THE V300 table

---------
<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h1>Homework #6</h1>
<h2><span class="fa fa-pencil"></span>The B-band data and making a CMD</h2>
</div>
</section>

> Your task in the homework is to repeat the steps above for the rest of your data, and make two CMDs, one for the 30s data, and one for the 300s data.

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-question"></span>  The B-band data (4 points)</h2>
</div>
</section>

> Repeat the steps above for the 300-second B-band data. Make a plot of the difference between instrumental B-band magnitude and UCAC4 B-band magnitude. Calculate the offset needed to correct your instrumental mags (zeropoint) and display it on your plot.

> Finally, add your zeropoint to the **Magnitude** column of the 300s B-band table

In [0]:
# YOUR CODE HERE

Now you should have a two astropy tables in computer memory. One with corrected (calibrated) V-band magnitudes and one with corrected (calibrated) B-band magnitudes. We need to cross-match these tables with each other, to find which stars in the V-band table match with stars in the B-band table. We can't use ```astroquery```'s Xmatch for this, since they are both local tables. Instead, we will use the ```SkyCoord``` object from astropy, which is meant to work with coordinates on the sky. You can create a ```SkyCoord``` object with positions of all of the stars in a table like so:

```python
from astropy.coordinates import SkyCoord

coo_v = SkyCoord(V300['CentroidRA'], V300['CentroidDec'], unit=u.deg)
```

The ```SkyCoord``` you made is a bit like a NumPy array, but it is an array of positions on the sky. It carries within it several useful functions, including one to match positions against another sky coordinate object - ```match_coordinates_sky```. It's usage is shown below, and the documentation is [here](http://docs.astropy.org/en/stable/coordinates/matchsep.html#matching-catalogs). ```match_coordinates_sky``` returns an array of indices and the 2D and 3D separations of the matches. The array of indices can be used as a slice to sort the second set of coordinates, or the table from which it came, so that each row of the sorted coordinate/table contains the closest match of the corresponding row. Perhaps an example will explain:

```python
# match every entry in coo_v with the nearest entry in coo_v
idx, distance_2d, distance_3d = coo_v.match_to_catalog_sky(coo_b)

# using idx as a slice for the table B300 will sort it so that B300[0] is the closest match to V300[0]
B300 = B300[idx]
```

We are not quite done yet though - we have found the closest match to every object in the V300 table, but some of those matches may be quite far away. We could create a *fancy slicing mask* of True and False values by comparing the ```distance_2d``` array to some threshold separation, and use that mask to remove the rows where there is no close cross-match:

```python
mask = distance_2d < 3 * u.arcsec
B300 = B300[mask]
V300 = V300[mask]
```

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-question"></span>Cross-matching your V and B-band tables (2 points)<h2>
</div>
</section>

> Create ```SkyCoord``` objects from your 300s B and V tables. Match them using ```match_coordinates_sky```, and use a mask to remove all rows where there is no good match.

In [0]:
# YOUR CODE HERE

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-question"></span>Plotting your CMD (3 points)</h2>
</div>
</section>

> With your matched tables in hand, extract the calibrated magnitude columns from each into two arrays called ```V``` and ```B```. Calculate ```B-V``` and plot a colour-magnitude diagram of ```B-V``` against ```V```. Your plot should look something like the one below, which is for the cluster NGC 7789.

<img src="../../images/CMD.png" style="margin: 0px" width=750px/>

> Also calculate error bars on ```B-V```: remember that $\Delta (x+y)^2 = \Delta x^2 + \Delta y^2$

> Hint: you can reverse the y-axis with ```plt.gca().invert_yaxis()```

In [0]:
# YOUR CODE HERE

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-question"></span>Save your data (1 point)</h2>
</div>
</section>

> Save your ```V``` and ```B-V``` data, along with their uncertainties, to a text file, for use in the next lab!


In [0]:
# YOUR CODE HERE